# Emotion Recognition Model Pipelines

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone git@github.com:aaptss/voice_emotion_classification.git
    %cd voice_emotion_classification

## Download data

In [ ]:
!test -d LCB_data_snd_devided || test -f LCB_data_snd_devided.zip || gdown http://headkraken.gg/soft/Dan/LCB_data_snd_devided.zip
!test -d LCB_data_snd_devided || unzip -qq LCB_data_snd_devided.zip

In [ ]:
!test -d demos || test -f LCB_demos.zip || gdown http://headkraken.gg/soft/LCB_demos.zip
!test -d demos || unzip -qq LCB_demos.zip

In [ ]:
!test -d LCB_added || test -f LCB_added_rep.zip || gdown --id 13s0uq6eWU5bx3Ctik5Wy4baN-wBctoOu
!test -d LCB_added || unzip -q LCB_added_rep.zip

In [ ]:
%pip install -q pydub

## Prepare data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from codes.data_prep import get_dataloader, get_train_test, BaseCustomDataset

In [ ]:
file_path='./LCB_added'
path_to_audio='./LCB_data_snd_devided'
path_to_splitted_audio='./LCB_audio'

In [ ]:
train_dataloader,val_dataloader = get_dataloader(file_path=file_path,
                                                 path_to_audio=path_to_audio,
                                                 path_to_splitted_audio=path_to_splitted_audio,
                                                 test_size=0.2,
                                                 use_game_context=True,
                                                 batch_size=128,
                                                 DatasetClass=AttentionLSTM_Dataset)

In [ ]:
data, ctx, labels = next(iter(train_dataloader))
print(data.shape, ctx.shape, labels.shape)

In [ ]:
%%time

for _ in train_dataloader:
    pass

In [ ]:
#for ML methods

# x_train,ctx_train,y_train,x_test,ctx_test,y_test = get_train_test(
#                                                         file_path=file_path,
#                                                         path_to_audio=path_to_audio,
#                                                         path_to_splitted_audio=path_to_splitted_audio,
#                                                         test_size=0.2,
#                                                         use_game_context=True
#                                                     )

In [ ]:
# x_train.shape,y_train.shape

## Model

In [ ]:
from codes import AttentionLSTM_Classifier, AttentionLSTM_Dataset, train

In [ ]:
if IN_COLAB:
    %load_ext tensorboard
    %tensorboard --logdir logs

In [ ]:
model = AttentionLSTM_Classifier(57, context_dims=0)

model, trainer = codes.base.train(model,
                    train_dataloader,
                    val_dataloader,
                    max_epochs=1)